<img src="https://user-images.githubusercontent.com/33928040/105023234-b5ed5900-5a70-11eb-9b3f-0f64afc99038.jpeg"  width="1000" height="200">


<h1><center>↫ Transfer Learning ↬</center></h1>

In [ ]:
pip install git+https://github.com/abhishekkrthakur/wtfml.git

In [ ]:
!pip install pretrainedmodels albumentations

In [ ]:
from IPython.core.display import HTML

# import Source Code Pro font
display(HTML("""
<style>
@import url('https://fonts.googleapis.com/css2?family=Source+Code+Pro&display=swap');
</style>
"""))

def css_styling():
    styles = open("../input/sloth-styling-css/alerts.css", "r").read()
    return HTML("<style>"+styles+"</style>")
css_styling()

# 1. Introduction

Hello everyone, in this notebook I want to share how I used <span style="background:#F2F2F2; font-weight:bold; color:#000000">PyTorch</span> for *transfer learning* for classifying dogs and cats. The code template is inspired by <span style="background:#F2F2F2; font-weight:normal; color:black"><a href="https://www.youtube.com/watch?v=WaCFd-vL4HA">this</a></span> YouTube tutorial. We will be using `se_resnext50_32x4d` model from the `pretrainedmodels` package.

<h2><span style="background:#F3E9D9; font-weight:normal; color:black; font-family: 'Source Code Pro'; monospace">Learning Objectives</span>📙</h2>

<ol>
    <li>How to use PyTorch for transfer learning?</li>
    <li>How to use joblib to parallelize resizing of images?</li>
</ol>

<h2><span style="background:#F3E9D9; font-weight:normal; color:black; font-family: 'Source Code Pro'; monospace">Prerequisite</span>🚨</h2>

Here are a few prerequisites. One should know:

<ol>
    <li>How to code in Python,</li>
    <li>The basics of ANN and CNN, and</li>
    <li>The basics of PyTorch</li>
</ol>

<h2><span style="background:#F3E9D9; font-weight:normal; color:black; font-family: 'Source Code Pro'; monospace">Data</span>💡</h2>

We are going to make use of the <span style="background:#F2F2F2; font-weight:normal; color:black"><a href="https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition">Dogs vs Cats</a></span> dataset.

The dataset contains dogs and cats images and our goal is to make a model that will classify images accurately.

# 2. Importing Libraries & Reading Data

<h2><span style="background:#F3E9D9; font-weight:normal; color:black; font-family: 'Source Code Pro'; monospace">Imports</span>🔛</h2>

Let us see all the required libraries we are going to use in this notebook:

<div class="alert success-alert">
  <p>NumPy🧮: for numerical computation.</p>
  <p>Pandas🐼: for data manipulation and analysis.</p>
  <p>Matplotlib📊: for creating visualizations.</p>
  <p>Joblib⚓: for providing lightweight pipelining in Python.</p>
  <p>Pillow🛌: for opening, manipulating, and saving many different image file formats.</p>
  <p>Tqdm🆒: for making smart progress meter.</p>
  <p>PyTorch🔦: for building artificial neural network.</p>
  <p>Scikit-Learn🤖: provides tools for predictive data analysis.</p>
  <p>Pretrainedmodels💪: provides pretrained ConvNets for pytorch.</p>
  <p>Albumentations🤸: for image augmentation</p>
  <p>WTFML🚀: for providing flexibility while using PyTorch.</p>
</div>

<br>

<h2><span style="background:#F3E9D9; font-weight:normal; color:black; font-family: 'Source Code Pro'; monospace">Datasets</span>💽</h2>

We have two datasets files:
    
<ol>
    <li><b>train.zip</b>: contains images that we will use for training and evaluation.</li>
    <li><b>test.zip</b>: contains images that we will use for submission.</li>
</ol>

> Note📌: There are no null values in either of the data set.

<b>👇 Version Number and Summary of Datasets</b>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rcParams

import sys, os, joblib, PIL, shutil
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

import sklearn
from sklearn import metrics

import pretrainedmodels, albumentations, wtfml
from wtfml.data_loaders.image.classification import ClassificationDataset
from wtfml.utils import EarlyStopping
from wtfml.engine import Engine

# default font-family
rcParams["font.family"] = "serif"

# set a default random seed
RANDOM_SEED = 42

# set pandas options
pd.set_option("display.max_columns", None)

# version used
print(f"🐍: {sys.version}")
print()
print("⟿ Version🔗:")
print()
print(f"    NumPy version: {np.__version__}")
print()
print(f"    Pandas version: {pd.__version__}")
print()
print(f"    Matplotlib version: {mpl.__version__}")
print()
print(f"    Joblib version: {joblib.__version__}")
print()
print(f"    Pillow version: {PIL.__version__}")
print()
print(f"    PyTorch version: {torch.__version__}")
print()
print(f"    Scikit-Learn version: {sklearn.__version__}")
print()
print(f"    PreTrainedModels version: {pretrainedmodels.__version__}")
print()
print(f"    Albumentations version: {albumentations.__version__}")
print()
print(f"    WTFML version: {wtfml.__version__}")

# Unzip The Data

* Since our train and test images are present inside a zip file, so our first step is to unzip all the images.

* We will unzip images to location: <span style="background:#F2F2F2; font-weight:normal; color:#000000">kaggle/temp</span>.

<b>✅ Click the toggle button on the right to access the code.</b>

In [ ]:
!unzip ../input/dogs-vs-cats-redux-kernels-edition/test.zip -d /kaggle/temp/
!unzip ../input/dogs-vs-cats-redux-kernels-edition/train.zip -d /kaggle/temp/

<h2><span style="background:#F3E9D9; font-weight:normal; color:black; font-family: 'Source Code Pro'; monospace">Navigating The Directory Tree</span>🌴</h2>

* As we have unzipped the images to their respective directories, let us see the images present inside our train directory.

> Note📌: The present working directory is <span style="background:#F2F2F2; font-weight:normal; color:#000000">kaggle/working</span>, and the images are saved in the location <span style="background:#F2F2F2; font-weight:normal; color:#000000">kaggle/temp/</span>

<b>👇 Current Working Directory and Number of Train Images</b>

In [ ]:
# train and test path
train_path = "../temp/train"
test_path = "../temp/test"

# total number of files present
print(f"The total number of images present: {len(os.listdir(train_path))}")

# total number of dogs and cats images
print(f"    ⟿ Total images of dogs: {len([file for file in os.listdir(train_path) if file[0:3] == 'dog'])}")
print(f"    ⟿ Total images of cats: {len([file for file in os.listdir(train_path) if file[0:3] == 'cat'])}")

<h2><span style="background:#F3E9D9; font-weight:normal; color:black; font-family: 'Source Code Pro'; monospace">Visualizing The Images</span>🆒</h2>

* Let's now define a function that will plot images for dogs and cats.

* The function will take a `value` (i.e. dog or cat or both), `path` (where the images are stored), and `title` (the title of the plot). It will plot 12 random images based on the `value` argument.

<b>👇 Dogs Images In Train-Directory</b>

In [ ]:
def plot_images(value, path, title):
    """
    Function to plot dogs/cats images.
    
    Args:
        value (str): either dog or cat or both.
        path (str): where images ae present.
        title (str, optional): title of the plot.
    
    Returns:
        figure.Figure: figure object.
        axes.Axes: axes object.
    """
    # create subplots
    fig, axes = plt.subplots(
        nrows = 3, ncols=4, facecolor="#FFFFFF",
        figsize=(8,6), dpi=450
    )
    
    # all images for given value
    if value != "both":
        images = [file for file in os.listdir(path) if file[0:3] == value]
    else:
        images = [file for file in os.listdir(path)]
        
    # select random numbers for indices
    random_index = np.random.randint(0, len(images) - 1, 12)

    # select 24 dogs images based on random_index
    select_images = np.array(images)[random_index]
    
    # traverse the axes and plot the images
    for count, ax in tqdm(enumerate(fig.get_axes()), desc="Plotting Images", total=len(select_images)):
        # set facecolor
        ax.set_facecolor("#FFFFFF")
        
        # read the image
        img = plt.imread(f"{path}/{select_images[count]}")
        
        # plot the image
        ax.imshow(img)
        
        # tidy axis
        ax.axis("off")
    
    # configure layout
    plt.tight_layout(pad=0.5)

    # add figure title
    fig.text(0.5, 1.07, title, fontsize=20, color="#121212", fontstyle="italic", ha="center")
    
    fig.subplots_adjust(hspace=0, wspace=0)
        
    return fig, axes

In [ ]:
# plot dogs images
fig, ax = plot_images("dog", train_path, "Woof! Woof!")

<b>👇 Cats Images In Train-Directory</b>

In [ ]:
# plot cats images
fig, ax = plot_images("cat", train_path, "Meow! Meow!")

# Split The Data

* There are `25000` images present inside the *train* directory. We are going to make three dataset out of all `25000` images:
    
    * `train-set`: our training set will include `22000` images (11000 dogs & 11000 cats).
    
    * `valid-set`: our validation set will include `1500` images (750 dogs & 750 cats).
    
    * `test-set`: our test set will include `1500` images (750 dogs & 750 cats).
    
* Here, we will prepare three lists which will contain image names for train, valid and test set.

<b>👇 Splits and Number of Images</b>

In [ ]:
# set random seed for numpy
np.random.seed(RANDOM_SEED)

# all dogs and cats images
dogs_images = sorted(np.array([file for file in os.listdir(train_path) if file[:3] == "dog"]))
cats_images = sorted(np.array([file for file in os.listdir(train_path) if file[:3] == "cat"]))

# shuffle both the array
np.random.shuffle(dogs_images)
np.random.shuffle(cats_images)

# start and end indices for train, valid and test sets
train_start, train_end = 0, 11000
valid_start, valid_end = 11000, 11750
test_start, test_end = 11750, 12501

# make train-set, valid-set and test-set
train_set = dogs_images[train_start: train_end] + cats_images[train_start: train_end]
valid_set = dogs_images[valid_start: valid_end] + cats_images[valid_start: valid_end]
test_set = dogs_images[test_start: test_end] + cats_images[test_start: test_end]

# shuffle all the data sets
np.random.shuffle(train_set)
np.random.shuffle(valid_set)
np.random.shuffle(test_set)

print("⟿ For Dogs:")
print(f"    Number of images in train-set: {len(dogs_images[train_start: train_end])}")
print(f"    Number of images in valid-set: {len(dogs_images[valid_start: valid_end])}")
print(f"    Number of images in test-set:  {len(dogs_images[test_start: test_end])}")
print()
print("⟿ For Cats:")
print(f"    Number of images in train-set: {len(cats_images[train_start: train_end])}")
print(f"    Number of images in valid-set: {len(cats_images[valid_start: valid_end])}")
print(f"    Number of images in test-set:  {len(cats_images[test_start: test_end])}")
print()
print("⟿ Final Datasets:")
print(f"    Length of train-set: {len(train_set)}")
print(f"    Length of valid-set: {len(valid_set)}")
print(f"    Length of test-set:  {len(test_set)}")

# Resize Images

* By default the `se_resnext50_32x4d` model takes input image <span style="background:#F2F2F2; font-weight:normal; color:black"><a href="https://github.com/Cadene/pretrained-models.pytorch/blob/master/pretrainedmodels/models/senet.py#L60">size</a></span> as `224x224`.

* Since, the image size in our dataset is not *224x224*, we now will resize the images to the required size. To speed up the task we are going to make use of `joblib` to *parallelize* the process.

* We are also going to make *four* new directories `train_set`, `valid_set`, `test_set` and `kaggle_test_set` containing resized images from `train` and `test` directories, and will delete the `train` and `test` directories we had initially.

<b>✅ Click the toggle button on the right to access the resize-function.</b>

In [ ]:
def resize_image(image_path, output_dir, resize):
    """
    Function to resize one image.
    
    Args:
        image_path (str): path to the image.
        output_dir (str): directory where final image will be stored.
        resize (tuple): required shape of the image.
    """
    # fetch the base name of the image
    base_name = os.path.basename(image_path)
    
    # set the output path
    out_path = os.path.join(output_dir, base_name)
    
    # open the image using pillow
    image = PIL.Image.open(image_path)
    
    # resize the image
    image = image.resize(
        (resize[0], resize[1]), resample=PIL.Image.BILINEAR
    )
    
    # save the image
    image.save(out_path)

<b>👇 Resizing Images From train Directory</b>

In [ ]:
# make three directories - for train, valid and test
os.mkdir("../temp/train_set")
os.mkdir("../temp/valid_set")
os.mkdir("../temp/test_set")

# resize the train-set images
joblib.Parallel(n_jobs=12)(
    joblib.delayed(resize_image)(
        f"{train_path}/{i}", "../temp/train_set", (224, 224)
    ) for _, i in tqdm(enumerate(train_set))
)

# resize the valid-set images
joblib.Parallel(n_jobs=12)(
    joblib.delayed(resize_image)(
        f"{train_path}/{i}", "../temp/valid_set", (224, 224)
    ) for _, i in tqdm(enumerate(valid_set))
)

# resize the test-set images
joblib.Parallel(n_jobs=12)(
    joblib.delayed(resize_image)(
        f"{train_path}/{i}", "../temp/test_set", (224, 224)
    ) for _, i in tqdm(enumerate(test_set))
)

# remove the train directory
shutil.rmtree(train_path)

<b>👇 Resizing Images From test Directory</b>

In [ ]:
# make directory for kaggle-test-set
os.mkdir("../temp/kaggle_test_set")

# fetch image names
kaggle_images = os.listdir(test_path)

# resize the test-set images
joblib.Parallel(n_jobs=12)(
    joblib.delayed(resize_image)(
        f"{test_path}/{i}", "../temp/kaggle_test_set", (224, 224)
    ) for _, i in tqdm(enumerate(kaggle_images))
)

# remove test1 directory
shutil.rmtree(test_path)

<b>👇 Resized Images From train_set Directory</b>

In [ ]:
# plot images
fig, ax = plot_images("both", "../temp/train_set", "Images in train-set")

<b>👇 Final Directories & Images Inside Them</b>

In [ ]:
dirs = os.listdir("../temp")

print(f"Directries present: {dirs}")
print()

print("⟿ Images Present:")

for dir_ in dirs:
    print(f"    In {dir_}: {len(os.listdir(f'../temp/{dir_}'))}")

# Using Pre-Trained Model

* We here are using `se_resnext50_32x4d` model from `pretrainedmodels` package. This model represents <span style="background:#F2F2F2; font-weight:normal; color:black">SE-ResNet</span> architecture and is trained on the <span style="background:#F2F2F2; font-weight:normal; color:black">Imagenet</span> dataset.

<h2><span style="background:#F3E9D9; font-weight:normal; color:black; font-family: 'Source Code Pro'; monospace">SENet</span>🔝</h2>

* *SENet* is the winning architecture in the ILSVRC 2017 challenge.

* SENet stands for <span style="background:#F2F2F2; font-weight:bold; color:black">Squeeze-and-Excitation Network</span>. This architecture extends other architectures such as `Inception Networks` and `Residual Networks` and boosts their performance.

* The extended versions of `Inception Networks` and `ResNets` are called `SE-inception` and `SE-ResNet` respectively.

* The boost comes from the fact that a SENet adds a small neural network, called an *SE block*, to every unit in the original architecture (i.e. every inception module or every residual unit).

<center>
<img src="https://user-images.githubusercontent.com/33928040/107626693-307e5280-6c84-11eb-82d6-23309d49e856.jpeg" height=500 width=1000>
</center>

<br>

* A SE block analyzes the output of the unit it is attached to, focusing exclusively on the depth dimension, and learns which features are mostly active together. It then uses this information to recalibrate the feature maps.

* For example, an SE block may learn that mouth, noses and eyes usually appear together in the pictures: if you see a mouth and a nose, you should expect to see eyes as well.

* So, if the block sees a strong activation in the mouth and nose feature maps, but only mild activation in the eye feature map, it will boost the eye feature map (more accurately, it will reduce irrelevant feature maps).

* If the eyes were somewhat confused with something else, this feature map recalibration will help resolve the ambiguity.

<h2><span style="background:#F3E9D9; font-weight:normal; color:black; font-family: 'Source Code Pro'; monospace">se_resnext50_32x4d</span>🏅</h2>

* We are going to use the <span style="background:#F2F2F2; font-weight:normal; color:black"><a href="https://github.com/Cadene/pretrained-models.pytorch/blob/master/pretrainedmodels/models/senet.py#L347">features</a></span> method defined in `SENet` class in `pretrainedmodels` package. This method returns `2048` feature values.

* We will then feed these `2048` feature values to our output layer which outputs a single floating number ranging between 0 to 1. (i.e. we will use `sigmoid` as our activation function for the output layer, `0` is `cat` and `1` is `dog`.

* To compute the loss we are using `Binary Cross Entropy` loss metric.

<b>👇 Download Required Model</b>

In [ ]:
class SEResNext50_32x4d(nn.Module):
    """
    class for our neural network architecture.
    """
    
    def __init__(self, pretrained="imagenet"):
        """
        Function to init object of the class.
        
        Args:
            pretrained (str): trained on which dataset.
        """
        # access the methods and features of parent class
        super(SEResNext50_32x4d, self).__init__()
        
        # create a base model
        self.model = pretrainedmodels.__dict__[
            "se_resnext50_32x4d"
        ](pretrained=pretrained)
        
        self.out = nn.Linear(
            in_features=2048, out_features=1
        )
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, image, targets):
        """
        Forwad propagation.
        
        Args:
            image (torch.tensor): batch of images.
            targets (torch.tensor): images-targets.
        
        Returns:
            float: final output.
        """
        batch_size, _, _, _ = image.shape
        
        # get features
        x = self.model.features(image)
        
        # perform adaptive pooling
        x = F.adaptive_avg_pool2d(x, 1)
        
        # reshape
        x = x.reshape(batch_size, -1)
        
        # get the output
        x = self.out(x)
        output = self.sigmoid(x)
        
        # compute loss
        loss = nn.BCELoss()(
            output, targets.reshape(-1, 1).type_as(output)
        )
        
        return output, loss

model = SEResNext50_32x4d()

<h2><span style="background:#F3E9D9; font-weight:normal; color:black; font-family: 'Source Code Pro'; monospace">Setting Default Values</span>📋</h2>

* We are going to use `50 epochs` and `batch-size` of `12`.

* Mean and Standard Deviation are used for normalizing the images, and for `se_resnext50_32x4d` mean and std can be found <span style="background:#F2F2F2; font-weight:normal; color:black"><a href="https://github.com/Cadene/pretrained-models.pytorch/blob/master/pretrainedmodels/models/senet.py#L60">here</a></span>.

* Since, Kaggle provides us with GPU so we are going to pass `device` as `"cuda"`.

<b>👇 Default Values</b>

In [ ]:
# default values
DEVICE = "cuda"
EPOCHS = 50
BATCH_SIZE = 32


mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

print("⟿ Default Values")
print(f"    Device: {DEVICE}")
print(f"    Epochs: {EPOCHS}")
print(f"    Batch-Size: {BATCH_SIZE}")
print()
print("⟿ Normalize Images")
print(f"    Mean: {mean}")
print(f"    Standard-Deviation: {std}")

<b>👇 Kaggle GPU Information</b>

In [ ]:
!nvidia-smi

# Let Us Train

* We will perform the following steps while training our model:
    
    * Use `albumentations` for making required normalization and augmentations.
    
    * Making `DataLoader` for train and valid set images using `wtfml` and `PyTorch`.
    
    * Initializing `optimizer` and `learning rate scheduler` using `PyTorch`.
    
    * Using `wtfml` to initialize `EarlyStopping` and `Engine` class oject. (To know more about what Engine is see this <span style="background:#F2F2F2; font-weight:normal; color:black"><a href="https://www.kaggle.com/slothfulwave612/pytorch-building-a-digit-recognizer#5.-Building-Artificial-Neural-Network">implementation</a></span>)
    
    * Finally iterate over epochs: train, validate and save the model.
    
<b>✅ Click the toggle button on the right to access the training function.</b>

In [ ]:
def run_training(model, train_set, valid_set):
    """
    Function to run the training process.
    
    Args:
        model (__main__.SEResNext50_32x4d): object of the model.
        train_set (list): list of train-image-names.
        valid_set (list): list of valid-image-names.
    """
    # path to train, validation and test data
    train_data_path = "../temp/train_set/"
    valid_data_path = "../temp/valid_set/"
    
    # path where model is saved
    model_path = "../temp/model.bin"
    
    # augmentation
    aug = albumentations.Compose(
        [
            albumentations.Normalize(
                mean, std, max_pixel_value=255.0, always_apply=True
            ),
            albumentations.augmentations.transforms.Flip(p=0.11)
        ]
    )
    
    # get the image path and target for train-set
    train_images = [train_data_path + image for image in train_set]
    train_targets = [1 if image[0:3] == "dog" else 0 for image in train_set]

    # get the image path and target for validation-set
    valid_images = [valid_data_path + image for image in valid_set]
    valid_targets = [1 if image[0:3] == "dog" else 0 for image in valid_set]
    
    # create ClassificationLoader object for train-images
    train_dataset = ClassificationDataset(
        image_paths=train_images, targets=train_targets,
        resize=None, augmentations=aug
    )
    
    # create DataLoader for train-images
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=BATCH_SIZE,
        shuffle=True, num_workers=4
    )
    
    # create ClassificationLoader object for valid-images
    valid_dataset = ClassificationDataset(
        image_paths=valid_images, targets=valid_targets,
        resize=None, augmentations=aug
    )
    
    # create DataLoader for valid-images
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=BATCH_SIZE,
        shuffle=False, num_workers=4
    )
    
    # transfer to device
    model.to(DEVICE)
    
    # init optimizer
    optimizer = torch.optim.Adam(
        model.parameters(), lr=1e-4
    )
    
    # init scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, patience=3, mode="max"
    )
    
    # init EarlyStopping object
    es = EarlyStopping(patience=5, mode="max")
    
    # init Engine object
    engine = Engine(
        model, optimizer, DEVICE
    )
    
    for epoch in range(EPOCHS):
        # train the model
        train_loss = engine.train(train_loader)
        
        # evaluate on validation set
        valid_loss, predictions = engine.evaluate(
            valid_loader, True
        )
        
        # fetch predictions
        predictions = np.vstack(
            ([tensor.cpu().numpy() for tensor in predictions])
        ).ravel()
        predictions = np.vstack((predictions)).ravel()
        predictions = [1 if value >= 0.5 else 0 for value in predictions]
        
        # get accuracy
        accuracy = metrics.accuracy_score(
            valid_targets, predictions
        )
        
        # update the parameter values
        scheduler.step(accuracy)
        
        print(f"Epoch={epoch}, Accuracy={accuracy}")
        
        # check for early stopping
        es(accuracy, model, model_path)
        
        if es.early_stop:
            # early stop when required
            print("Early Stopping")
            break

run_training(model, train_set, valid_set)

<h2><span style="background:#F3E9D9; font-weight:normal; color:black; font-family: 'Source Code Pro'; monospace">Load The Model</span>📥</h2>

* The best performing model was giving aroung `99.4%` accuracy on the validation set.

* Since we have saved the best performing model during the training process, we can now load it to use it for checking the scores in the test set.

<b>✅ Click the toggle button on the right to access the code.</b>

In [ ]:
# load the best model
best_model = SEResNext50_32x4d()
best_model.load_state_dict(torch.load("../temp/model.bin"))

# Test Scores

* We will use the best performing model on our test set to check the final accuracy. 

* We again have to perform a few steps as we did in the training function to get our test-score.

<b>👇 Accuracy on Test Set</b>

In [ ]:
def eval_test(model, test_set, test_path, test_targets=None, get_accuracy=True):
    # transfer model to GPU
    model.to("cuda")
    
    # init Engine object
    engine = Engine(
        model, None, DEVICE
    )

    # augmentation
    aug = albumentations.Compose(
        [
            albumentations.Normalize(
                mean, std, max_pixel_value=255.0, always_apply=True
            )
        ]
    )

    # get the image path and target for validation-set
    test_images = [test_path + image for image in test_set]
    
    if test_targets is None:
        test_targets = [1 if image[0:3] == "dog" else 0 for image in test_set]

    # create ClassificationLoader object for test-images
    test_dataset = ClassificationDataset(
        image_paths=test_images, targets=test_targets, 
        resize=None, augmentations=aug
    )

    # create DataLoader for test-images
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=BATCH_SIZE,
        shuffle=False, num_workers=4
    )

    # check on test set
    predictions = engine.predict(test_loader)  
        
    # fetch probability
    predictions = np.vstack(
        ([tensor.cpu().numpy() for tensor in predictions])
    ).ravel()
    
    if get_accuracy:
        # get predictions
        predictions = [1 if value >= 0.5 else 0 for value in predictions]
        
        # compute accuracy
        accuracy = metrics.accuracy_score(
            test_targets, predictions
        )

        print(f"Accuracy on test-set: {accuracy}")
        return
        
    return predictions

eval_test(best_model, test_set, "../temp/test_set/", test_targets=None, get_accuracy=True)

<h2><span style="background:#F3E9D9; font-weight:normal; color:black; font-family: 'Source Code Pro'; monospace">Final Submission</span>🏁</h2>

* We will use the function defined above to get the predictions for the `kaggle_test_set` images.

* At last we will create the required dataframe containing `image-id` and it's corresponding `probability`.

<b>👇 Prediction on kaggle-set.</b>

In [ ]:
preds = eval_test(
    best_model, kaggle_images, "../temp/kaggle_test_set/", test_targets=[0] * len(kaggle_images), get_accuracy=False
)

<b>👇 Saving The Dataframe</b>

In [ ]:
# make dataframe
df = pd.DataFrame(
    {
        "id": [int(image[:-4]) for image in kaggle_images], 
        "label": preds
    }
)

# sort values
df = df.sort_values(by="id").reset_index(drop=True)

# save the dataframe
df.to_csv("predictions.csv", index=False)

print("Saved!!!")

So, that's everything I want to share through this notebook. I hope you have enjoyed and learnt something, and if you like my efforts do give an upvote to this notebook and share it.

Thank you for your presence.

<h3><span style="background:#F3E9D9; font-weight:normal; color:black; font-family: 'Source Code Pro'; monospace">Footnote</span>📍</h3>

<ol>
    <li>The code template is inspired by <span style="background:#F2F2F2; font-weight:normal; color:black"><a href="https://www.kaggle.com/abhishek">Abhisekh Thakur</a></span>.</li>
    <li>The notebook theme is inspired by <span style="background:#F2F2F2; font-weight:normal; color:black"><a href="https://www.kaggle.com/andradaolteanu">Andrada Olteanu</a></span>.</li>
</ol>